## Install
run dataset for every run the version in Colab is not working for importing datasets

In [1]:
%pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [11]:
import torch
import torchdata
import transformers
from datasets import load_dataset

from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import huggingface_hub

dash_line = '_'.join('' for x in range(100))

In [3]:
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

## good representation of the dataset



In [4]:


example_indices = [40,200]

dash_line = '_'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(dash_line)
  print('Example', i+1)
  print(dash_line)
  print('Input')
  print(ds['test'][index]['dialogue'])
  print(dash_line)
  print('Baseline Human summary')
  print(ds['test'][index]['summary'])
  print()

___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
Input
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
Baseline Human summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

___________________________________________________________________________________________________
Example 2
___________________________________________________________________________________________________
Input

## model import and tokenizer implementation

google flan-t5

In [22]:
model_name = 'google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [24]:
sentence ='What time is it, Tom?'

sentence_encoded = tokenizer(sentence, return_tensors='pt')


sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0], skip_special_tokens= True)


print(sentence_encoded['input_ids'][0])
print(sentence_decoded)

tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])
What time is it, Tom?


In [25]:
## random word decoding - for fun
tensor1=torch.tensor([563, 19])

sentence_decoded1 = tokenizer.decode(tensor1, skip_special_tokens= True)
print(sentence_decoded1)

group is


In [27]:
dialogue = ds['test'][41]['dialogue']
print(dialogue)
summary = ds['test'][41]['summary']

input = tokenizer(dialogue, return_tensors='pt')
input_tensor = input['input_ids']

output= model.generate(input_tensor, max_new_tokens = 500)


#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.


In [28]:
print(output)
decoded_output = tokenizer.decode(output[0], skip_special_tokens= True)
print('model summary : ', decoded_output)
print('baseline summary: ',summary)

tensor([[   0, 5780,  536,   10,   94,   31,    7,    3,  324,   12, 4169,    5,
            1]])
model summary :  Person1: It's ten to nine.
baseline summary:  #Person1# is rushing to catch a train but Tom thinks it isn't necessary.


## Zero shot inference
instructing the model to perform a task

In [31]:
## good representation here

example_indices = [40,42]



for i, index in enumerate(example_indices):
  dialogue = ds['test'][index]['dialogue']
  summary = ds['test'][index]['summary']
  prompt = f"""

  summarize the following conversation :
  {dialogue}

  Summary:
  """

  input = tokenizer(prompt, return_tensors='pt')
  input_tensor = input['input_ids']

  output= model.generate(input_tensor, max_new_tokens = 50)
  decoded_output = tokenizer.decode(output[0], skip_special_tokens= True)
  print(dash_line)
  print('Example', i+1)
  print(dash_line)
  print('Input')
  print(ds['test'][index]['dialogue'])
  print(dash_line)
  print('Zero shot model summary ')
  print(decoded_output)
  print('Baseline Human summary')
  print(summary)
  print()

___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
Input
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
Zero shot model summary 
The train is about to leave.
Baseline Human summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

___________________________________________________________________________________________________
Example 2
___________________________________________________

another example here for zero shot inference

In [32]:


example_indices = [40,42]

dash_line = '_'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  dialogue = ds['test'][index]['dialogue']
  summary = ds['test'][index]['summary']
  prompt = f"""

  summarize the following conversation :
  {dialogue}

  what is going on here, tell what is happening:
  """

  input = tokenizer(prompt, return_tensors='pt')
  input_tensor = input['input_ids']

  output= model.generate(input_tensor, max_new_tokens = 50)
  decoded_output = tokenizer.decode(output[0], skip_special_tokens= True)
  print(dash_line)
  print('Example', i+1)
  print(dash_line)
  print('Input')
  print(ds['test'][index]['dialogue'])
  print(dash_line, '\n')
  print('Zero shot model summary ')
  print(decoded_output,'\n')
  print('Baseline Human summary')
  print(summary)
  print()

___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
Input
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________ 

Zero shot model summary 
#Person1: It's nine thirty. 

Baseline Human summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

___________________________________________________________________________________________________
Example 2
________________________________________________

## one shot and few shot inference

In [33]:
def make_fewshot_prompt(example_indices, test_indices):
  prompt =''

  for i, index  in enumerate(example_indices):
    dialogue = ds['test'][index]['dialogue']
    summary = ds['test'][index]['summary']
    prompt += f"""

    summarize :
    {dialogue}

    Summary, tell what is going on:
    {summary}
    """

  dialogue = ds['test'][test_indices]['dialogue']

  prompt += f"""

    summarize  :
    {dialogue}

    Summary, tell what is going on:

    """
  return prompt


In [34]:
example_indices = [40]

test_indices=300
fS_prompt=make_fewshot_prompt(example_indices, test_indices)
print(fS_prompt)



    summarize :
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there. 

    Summary, tell what is going on:
    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
    

    summarize  :
    #Person1#: I cannot imagine if Trump were to be our President again.
#Person2#: I am proud to say that he is our President, and I will be really happy if he could be re-elected.
#Person1#: You voted for him, right?
#Person2#: Did you vote for him, because I know that I did.
#Person1#: I am not sure about this.
#Person2#: I have nothing but faith in Trump.
#Person1#: What?
#Person2#: I am pretty sure he will make America great again!
#Person1#: Well, though we 

In [29]:
input = tokenizer(fS_prompt, return_tensors='pt')
input_tensor = input['input_ids']

output= model.generate(input_tensor, max_new_tokens = 50)
decoded_output = tokenizer.decode(output[0], skip_special_tokens= True)

summary = ds['test'][test_indices]['summary']




In [30]:
print('Input')
print(ds['test'][test_indices]['dialogue'])
print(dash_line, '\n')
print('Few shot model summary ')
print(decoded_output,'\n')
print('Baseline Human summary')
print(summary)
print()

Input
#Person1#: I cannot imagine if Trump were to be our President again.
#Person2#: I am proud to say that he is our President, and I will be really happy if he could be re-elected.
#Person1#: You voted for him, right?
#Person2#: Did you vote for him, because I know that I did.
#Person1#: I am not sure about this.
#Person2#: I have nothing but faith in Trump.
#Person1#: What?
#Person2#: I am pretty sure he will make America great again!
#Person1#: Well, though we do need some change in this country, I don't think he is the right person.
#Person2#: Our country is already changing as it is.
#Person1#: You are right about this.
#Person2#: I trust that he will take good care of our country.
#Person1#: Well, I don't think so. I will vote for Biden anyway.
___________________________________________________________________________________________________ 

Few shot model summary 
#Person1: I cannot imagine if Trump were to be our President again. #Person2: I am proud to say that he is our 